In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

import numpy as np
import torch

import sys
sys.path.append('../tensnet/src/')
import tensnet as tt

torch.set_default_dtype(torch.float64)

Print slice of solution and TOT number of parameters from 3D QTT solver evolution

In [ ]:
Li = 10
d = 2
encoding_prefix = "sta"
opt_method = "linsys"
noq = 3 * Li
bd_max = 100
T = 20

var = 1729495637
name = f'{encoding_prefix}_{opt_method}_noq_{noq}_BD_{bd_max}_iter_{T}_{var}'


solution = tt.load_object(f'/Users/stefanopisoni/NoSync/3D_evolution/results/{name}.mpssol')
slice_mps = solution[0][10] # first index: u,v,w,p,times; second index: snapshot from 0 to T-1
print(slice_mps)
total_params = sum(t.numel() for t in slice_mps.parameters())
print(total_params)

Print TOT number of parameters from compressed MPS (torch)

In [ ]:
bd_max = 100

solution = torch.load(f'/Users/stefanopisoni/NoSync/MPS/10qubits_torch/tur_u_mps_sta_BD_{bd_max}_torch.mps')
print(solution)
total_params = sum(t.numel() for t in solution.parameters())
print(total_params)

$L^{\infty}$-norm of the solution, computed to evaluate CFL.

In [ ]:
vec = slice_mps.contract()
max_abs_value = torch.max(torch.abs(vec))
print(max_abs_value)

Print slices from 3D QTT evolution

In [ ]:
slice_mps_9 = slice_mps.average(9).average(18).average(27)
slice_mps_8 = slice_mps_9.average(8).average(16).average(24)

slice_vec_8 = slice_mps_8.contract().reshape(2**8,2**8,2**8)
print(slice_vec_8.shape)

# Print figure
fig = plt.figure(figsize=(12,12))
a = fig.add_subplot(121)
p = a.imshow(slice_vec_8[:,:,100])
a.set_title('Slice of averaged solution on 8qb for 3D QTT solver')

cbar1 = fig.colorbar(p, ax=a, fraction=0.046, pad=0.04)
cbar1.ax.tick_params(labelsize=8)

plt.savefig('Slice of averaged solution on 8qb for 3D QTT solver.pdf')